# Training, Testing and Predicting With an LSTM Model
In the notebook `ml/deeplearning_tuning.ipynb` we performed a Bayesian hyperparameter tuning to find the best hyperparameters for 3 potential LSTM models architectures.
In this notebook we:
1. train each of the 3 LSTM architectures with their best hyperparameters
2. tested their results on the test dataset to select the best model
3. analyze the sensibility of the best model to hyperparameters
4. use the best model to predict the groundwater depth for the year 2022.
## Multiple Multivariate Time Series Predictions with LSTM
The dataset is made of 478 Township-Ranges, each containing a multivariate (81 features) time series (data between 2014 to 2021). This dataset can thus be seen as a 3 dimensional dataset of
$478\ TownshipRanges\ *\ 8\ time stamps\ *\ 81\ features$
The objective is to predict the 2022 target value of `GSE_GWE` (Ground Surface Elevation to Groundwater Water Elevation - Depth to groundwater elevation in feet below ground surface) for each Township-Range.

LSTMs are used for time series and NLP because they are both sequential data and depend on previous states.
The future prediction *Y(t+n+1)* depends not only on the last state *X1(t+n), ..., Y(t+n)*, not only on past values of the feature *Y(t+1), ..., Y(t+n)*, but on the entire past states sequence.

![Multi-Variate Multi TImes-Series Predictions with LSTM - Training and Prediction](../doc/images/lstm_inputs_outputs.jpg)

During training and predictions:
* Township-Ranges are passed into the model one by one
* each cell in the LSM neural network receives a Township-Range state for a specific year (the state of the Township-Range at a specific position in the series)
* each state (year) in the series is represented by a multidimensional vector of all 81 features (including the target feature Y `GSE_GWE`)

The output is the Township-Ranges next year's value for the specific feature Y `GSE_GWE`. The model is trained on 2014-2020 (7 years) data to predict 2021.
During inference the last 7 years (2015-2021) of data are passed as input to predict the 2022 value.

![Multi-Variate Multi TImes-Series Predictions with LSTM - Cells Inputs](../doc/images/lstm_table_to_cells.jpg)

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import random
import tensorflow

from sklearn import set_config
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, RepeatVector
from keras.optimizers import Adam

from lib.township_range import TownshipRanges
from lib.read_data import read_and_join_output_file
from lib.deeplearning import get_train_test_datasets, get_sets_shapes, get_data_for_prediction, evaluate_forecast, combine_all_target_years, get_year_to_year_differences
from lib.viz import view_trs_side_by_side, draw_hyperparameters_distribution

In [ ]:
RANDOM_SEED = 31
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tensorflow.random.set_seed(RANDOM_SEED)

In [ ]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

## Preparing the Dataset
### The Train-Test Split
To fit our dataset and objective, as well as LSTM neural networks architecture we perform the train test split as follows:
* Training and Test sets will be split by Township-Ranges. I.e., some Township-Ranges will have all their 2014-2021 data points in the training set, some others will be in the test set.
* The model will be trained based on the 2014-2020 data for all features - including the target feature - with the training target being the 2021 value of the target feature.

With such a method, unlike a simple time series forecasting where the target feature is forecasted only based on its past value, we allow past value of other features (in our case cultivated crops, precipitations, population density, number of wells drilled) to influence the future value of the target feature.

![Train-Test Split](../doc/images/lstm-train-test-split.jpg)

We do not create a validation dataset as we use Keras internal cross-validation mechanism to shuffle the data points (i.e., the Township-Ranges) and keep some for the validation at each training epoch.
### Data Imputation and Scaling
Missing data imputation for a Township-Range is performed only using the existing data of that Township-Range (not the data of all Township-Ranges). For example:
* a *fill forward* approach is used for many fields like crops, vegetation and soils. The percentage of land use per crop in 2014 in a Township-Range is imputed into the missing year 2015 for that particular Township-Range.
* for fields like `PCT_OF_CAPACITY` (the capacity percentage of water reservoir), missing values in a Township-Range are filled using the min, mean, median or max values of that particular Township-Range. In this case the *fit* method of our custom impute pipeline does nothing. Not only the impute pipeline does not need to learn values from other Township-Ranges data points to impute missing values in a Township-Range, if it does it will not be able to impute data in the test set as we impute by Township-Range and the Township-Ranges in the test set are not seen when *fitting* the impute pipeline. The *transform* method, then simply fills the missing values in a Township-Range based on past values of that Township range. This way, we can split the train and test sets by Township-Range and impute missing value without any data leakage as the impute pipeline does not learn anything from the Township-Ranges in the test set.

We use a MinMax scaler to scale all values between 0 and 1 for the neural network, except for the vegetation, soils and crops datasets which are already scaled between 0 and 1.

It should be noted that we do not need to do any data imputation on the training and test sets *y* target feature since it does not have any missing data point.

In [ ]:
test_size=0.15
target_variable="GSE_GWE"
# Load the data from the ETL output files
X = read_and_join_output_file()
# Split the input pandas Dataframe into training and test datasets, applies the impute pipeline
# transformation and reshapes the datasets to 3D (samples, time, features) numpy arrays
X_train, X_test, y_train, y_test, impute_pipeline, impute_columns, target_scaler = get_train_test_datasets(X, target_variable=target_variable,
                                                                                           test_size=test_size, random_seed=RANDOM_SEED, save_to_file=True)
model_predictions_df = pd.DataFrame(y_test, columns=[target_variable])
model_scores_df = pd.DataFrame(columns=["mae", "mse", "rmse"])
nb_features = X_train.shape[-1]
get_sets_shapes(X_train, X_test)

In [ ]:
set_config(display="diagram")
display(impute_pipeline)

## Training Different Models
We tried 3 different LSTM models:
* A simple model made of a single *LSTM* layer and an output *Dense* layer
* A model made of a *LSTM* layer followed by a *Dense* and *Dropout* layers before the output layer
* An Encoder-Decoder model made of 2 *LSTM* layers followed by a *Dense* and *Dropout* layers

![LSTM Model Architectures](../doc/images/lstm_architectures.jpg)


Encoder-decoder architectures are more common for sequence-to-sequence learning e.g., when forecasting the next 3 days (output sequence of length 3) based on the past year data (input sequence of length 365). In our case we only predict data for 1 time step in the feature. The output sequence being of length 1 this architecture might seem superfluous but has been tested anyway. This architecture was inspired by the Encoder-Decoder architecture in this article: *[CNN-LSTM-Based Models for Multiple Parallel Input and Multi-Step Forecast](https://towardsdatascience.com/cnn-lstm-based-models-for-multiple-parallel-input-and-multi-step-forecast-6fe2172f7668)*.

As such models are made for sequence to sequence learning and forecasting, the output of such a model is different from the previous ones. It has an output of size *[samples, forecasting sequence length, target features]*. In our case the forecasting sequence length and number of target features are both 1.
### Training Model 1 - Simple LSTM Model

In [ ]:
m1_hyper_parameters = {
    "lstm_units": 160,
    "lstm_activation": "sigmoid",
    "learning_rate": 0.001,
    "validation_split": 0.1,
    "batch_size": 128,
    "epochs": 270,
}

model1 = Sequential()
model1.add(LSTM(m1_hyper_parameters["lstm_units"], activation=m1_hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model1.add(Dense(1, activation="linear"))
model1.summary()

In [ ]:
model1.compile(loss="mse", optimizer=Adam(learning_rate=m1_hyper_parameters["learning_rate"]), metrics=[keras.metrics.RootMeanSquaredError()])
model1.fit(X_train, y_train,
           validation_split=m1_hyper_parameters["validation_split"],
           batch_size=m1_hyper_parameters["batch_size"],
           epochs=m1_hyper_parameters["epochs"],
           shuffle=True)
yhat = model1.predict(X_test, verbose=0)
yhat_inverse = target_scaler.inverse_transform(yhat)
model_predictions_df["model_1_prediction"] = yhat_inverse
model_scores_df.loc["model 1"] = evaluate_forecast(y_test, yhat_inverse)

### Training Model 2 - LSTM + Dense Layer Model

In [ ]:
m2_hyper_parameters = {
    "lstm_units": 100,
    "lstm_activation": "sigmoid",
    "dense_units": 11,
    "dense_activation": "tanh",
    "dropout": 0.1,
    "learning_rate": 0.0001,
    "validation_split": 0.1,
    "batch_size": 32,
    "epochs": 200,
}

model2 = Sequential()
model2.add(LSTM(m2_hyper_parameters["lstm_units"], activation=m2_hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model2.add(Dense(m2_hyper_parameters["dense_units"], activation=m2_hyper_parameters["dense_activation"]))
model2.add(Dropout(m2_hyper_parameters["dropout"]))
model2.add(Dense(1, activation="linear"))
model2.summary()

In [ ]:
model2.compile(loss="mse", optimizer=Adam(learning_rate=m2_hyper_parameters["learning_rate"]), metrics=[keras.metrics.RootMeanSquaredError()])
model2.fit(X_train, y_train,
           validation_split=m2_hyper_parameters["validation_split"],
           batch_size=m2_hyper_parameters["batch_size"],
           epochs=m2_hyper_parameters["epochs"],
           shuffle=True)
yhat = model2.predict(X_test, verbose=0)
yhat_inverse = target_scaler.inverse_transform(yhat)
model_predictions_df["model_2_prediction"] = yhat_inverse
model_scores_df.loc["model 2"] = evaluate_forecast(y_test, yhat_inverse)

### Training Model 3 - Encoder-Decoder LSTM Model

In [ ]:
m3_hyper_parameters = {
    "lstm_units": 300,
    "lstm_activation": "sigmoid",
    "2nd_lstm_units": 140,
    "2nd_lstm_activation": "sigmoid",
    "dense_units": 21,
    "dense_activation": "tanh",
    "dropout": 0.2,
    "learning_rate": 0.001,
    "validation_split": 0.1,
    "batch_size": 32,
    "epochs": 200,
}


model3 = Sequential()
model3.add(LSTM(m3_hyper_parameters["lstm_units"], activation=m3_hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model3.add(RepeatVector(1))
model3.add(LSTM(m3_hyper_parameters["2nd_lstm_units"], activation=m3_hyper_parameters["lstm_activation"], return_sequences=True))
model3.add(TimeDistributed(Dense(m3_hyper_parameters["dense_units"], activation=m3_hyper_parameters["dense_activation"])))
model3.add(Dropout(m3_hyper_parameters["dropout"]))
model3.add(Dense(1, activation="linear"))
model3.summary()

In [ ]:
y_train_3d =  y_train[..., np.newaxis]
model3.compile(loss="mse", optimizer=Adam(learning_rate=m3_hyper_parameters["learning_rate"]), metrics=[keras.metrics.RootMeanSquaredError()])
model3.fit(X_train, y_train_3d,
           validation_split=m3_hyper_parameters["validation_split"],
           batch_size=m3_hyper_parameters["batch_size"],
           epochs=m3_hyper_parameters["epochs"],
           shuffle=True)
yhat = model3.predict(X_test, verbose=0)
yhat_inverse = target_scaler.inverse_transform(yhat.squeeze(2))
model_predictions_df["model_3_prediction"] = yhat_inverse
model_scores_df.loc["model 3"] = evaluate_forecast(y_test, yhat_inverse)

## Comparing the Different Models
### Comparing the Model Scores

In [ ]:
model_scores_df

### Comparing the Model Predictions on the Test Dataset
Here we are comparing the target variable values for the year 2021 for the Township-Ranges in the test set compared to the prediction made by each model based on the 2014-2020 data for the Township-Ranges in the test set.

In [ ]:
model_predictions_df

Based on the model scores it turns out that the simplest of the three LSTM models is the one having the best scores.

However, considering all the measurements between 2014 and 2022, the `GSE_GWE` (Ground Surface Elevation to Groundwater Water Elevation - Depth to groundwater elevation in feet below ground surface) target value has a
* median of 137.09 (~41.7 meters)
* mean value of 167.37 feet (~50.9 meters)
* min value of 0.5 feet (0 meters)
* max value of 727.5 feet (221.6 meters)

A mean average error of 23.66 feet (7.2 meters), and root mean square error of 34.82 feet (10.6 meters) in the prediction is fairly large. Even the best model does not seem to be accurate enough to be useful.

We save the best model anyway to perform predictions and analyze the results. Refer to the notebook `/ml/deeplearning_results.ipynb` for the analysis of the 2022 predictions results.

In [ ]:
model_dir = "../assets/models/"
keras_model_dir = os.path.join(model_dir, "keras_lstm_model")
os.makedirs(keras_model_dir, exist_ok=True)
# Save the Keras Model
model1.save(keras_model_dir)
# Save the data imputation pipeline and target min-max scaler
pipeline_data = {
    "impute_pipeline": impute_pipeline,
    "impute_columns": impute_columns,
    "target_scaler": target_scaler
}
with open(os.path.join(model_dir, "lstm_model_pipeline.pkl"), "wb") as file:
    pickle.dump(pipeline_data, file)

## Sensitivity Analysis
We perform here an analysis of the best model's sensitivity to the following hyperparamters:
* the optimizer used (e.g. Adam RMSprop, Adagrad)
* the training validation datasets split
* the number of lstm units
* the learning rate
* the batch size
* the number of training epochs

To perform this analysis, we trained 33,345 LSTM models for all possible combinations (within the selected ranges of values) of those 6 hyperparameters on the best model, and recorded for each model, the Root Mean Square Error (RMSE) on the test set.
The results are stored in a CSV file available in the ../assets/tuning folder.

The below visualization displays for each hyperparameter value, the distribution of the RMSE, and the mean of RMSE (using the color), for all models trained with that hyperparameter value. This allows us to show if a specific hyperparameter tends to lead to lower or higher RMSE and to compare the distribution between two values of the same hyperparameters.

In [ ]:
hpt_df = pd.read_csv(r"../assets/tuning/hpt_results.csv")
# We can discard some hyperparameter data to reduce the size of the visualization and improve readability
#hpt_df = hpt_df[hpt_df["epochs"].isin(range(50, 310, 40))]
#hpt_df = hpt_df[hpt_df["lstm_units"].isin(range(10, 200, 30))]
draw_hyperparameters_distribution(hpt_df, ["optimizer", "validation_split", "learning_rate", "batch_size", "epochs", "lstm_units"])

Looking at this visualization, we can see, with some surprise, that the hyperparameters which seem to have the biggest impact on the model performance have little to do with the model architecture itself (the number of LSTM units) but with how the model is trained.
* The choice of the optimizer seems to have the largest impact on the model performance, with both the mean and distribution of the RMSE for all models trained with an `Adagrad` optimizer being really bad.
* The training-validation percentage split seems to have little impact. Assign 5 or 10 % of the training data to validation seems to affect very little the performance of the trained models. But above 10% the performance starts to deteriorate.
* The bigger the learning rate, the better the models performs in terms or RMSE. The distribution of all models RMSE shows that with a learning rate of 0.1, most models have low RMSE around 60, while with a learning rate of 0.0001, the distribution of the RMSE of the models is more evenly spread across all values
* On the other hand, the smaller the batch size, the more models have a low RMSE.
* Although there is less of a difference if we compare similar values (e.g. 50 and 70 epochs or 270 and 290' epochs), we still see clearly that the bigger the number training epochs the more there are trained models with a low RMSE.
* The number of LSTM units, impacting the number of neurons in the LSTM model, seems to have less impact on the performance of the RMSE. The distribution of all models RMSE doesn't change much. But we see that the mean RMSE does slightly drop when the number of LSTM units increases.
## Predicting 2022
Even though our best model has a too large error to be useful, we can try as an exercise, to predict the 2022 target variable for all the Township-Ranges.

The model was trained to predict the 2021 data based on the previous 7 years of data 2014 to 2020. To predict 2022 we thus need to pass the previous 7 years of data (2015-2021). To do so:
1. We use our impute pipeline trained on the training dataset to impute values on the entire dataset and normalize the data
2. We drop the 2014 data points
3. We reshape the dataset as a 3 dimensional numpy array in the form of [all Township-Ranges, 2015-2021, 81 features]

Once we predict the 2022 values of the target variable, we extract the 2021 values from the original dataset to compare the 2021 values with the predicted 2022 values.

In [ ]:
# Predict the 2022 values for all Township-Ranges for the target variable based on 2015-2021 data
X_2015_to_2021 = get_data_for_prediction(X, impute_pipeline, impute_columns)
yhat_2022 = model1.predict(X_2015_to_2021, verbose=0)
yhat_inverse_2022 = target_scaler.inverse_transform(yhat_2022)
predictions_2022_df = pd.DataFrame(yhat_inverse_2022, index=X.index.get_level_values(0).unique(), columns=[target_variable])
# Add the 2022 values of the target variable to the existing ones
all_years_df = combine_all_target_years(X, target_variable, predictions_2022_df)
all_years_df

In [ ]:
township_range = TownshipRanges()
all_years_map_df = pd.merge(township_range.sjv_township_range_df, all_years_df.reset_index(), how="left", on=["TOWNSHIP_RANGE", ])
view_trs_side_by_side(all_years_map_df, feature="YEAR", value="GSE_GWE", title="San Joaquin Valley GSE_GWE with 2022 predictions")

When we look at the 2022 predictions of the `GSE_GWE` compared to the actual 2014-2021 the predictions are fairly consistent. Areas with high `GSE_GWE` values (i.e., deep ground to water depth) remain the same, and area with low `GSE_GWE` remain the same. However, if the model follows the past year trend, even with a high RMSE of 34.82 feet (10.6 meters), areas with high `GSE_GWE` will remain areas of high `GSE_GWE`. Comparing the past years `GSE_GWE` measurement values with the 2022 predictions is thus only partly informative about the quality of the prediction.

We thus try to also compare the year-to-year *difference* in the `GSE_GWE` from 2014 to 2021 and between our 2022 predictions and the 2021 values.

In [ ]:
yty_difference_df = get_year_to_year_differences(X, target_variable, predictions_2022_df)
yty_difference_df

In [ ]:
difference_df = pd.merge(township_range.sjv_township_range_df, pd.melt(yty_difference_df.reset_index(), id_vars=["TOWNSHIP_RANGE"], var_name="YEAR", value_name="GSE_GWE_DIFFERENCE"), how="left", on=["TOWNSHIP_RANGE", ])
view_trs_side_by_side(difference_df, feature="YEAR", value="GSE_GWE_DIFFERENCE", title="San Joaquin Valley GSE_GWE year-to-year variations from 2014 until 2022 predictions")

In [ ]:
yty_difference_df.describe()

Looking at the above table, here too, the difference between the 2022 predictions and 2021 measurements of `GSE_GWE` remains consistent with the year-to-year difference from previous years. Despite the RMSE score being bad, the year-to-year variation of `GSE_GWE` remains within acceptable range.
## Conclusion
Using a simple LSTM neural network to make next year predictions based on the past 7 years of data, we are able to achieve a more accurate prediction on the test set with an RMSE of 34.82 feet (10.6 meters) compared to an RMSE between 75 and 95 feet (22.8 and 28.9 meters) using supervised algorithms like XGBoost or K-Neighbors regressor.

The 2022 predictions look to be within the range of acceptable values and year-to-year variations. However, if the objective is to help policymakers and water resources management agencies predict a year in advance where to focus their attention in terms of well water shortages and drilling, the level of error of the model feels too big to be useful.